In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/bn_30m.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2023, 1, 19),
 datetime.date(2019, 12, 20),
 datetime.date(2024, 6, 12),
 datetime.date(2018, 4, 27),
 datetime.date(2018, 7, 10),
 datetime.date(2018, 10, 8),
 datetime.date(2024, 7, 5),
 datetime.date(2022, 6, 16),
 datetime.date(2019, 8, 30),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 3, 30),
 datetime.date(2018, 8, 6),
 datetime.date(2022, 11, 28),
 datetime.date(2018, 9, 17),
 datetime.date(2020, 8, 19),
 datetime.date(2021, 7, 5),
 datetime.date(2022, 6, 15),
 datetime.date(2019, 2, 6),
 datetime.date(2019, 3, 11),
 datetime.date(2020, 12, 23),
 datetime.date(2024, 3, 27),
 datetime.date(2018, 10, 23),
 datetime.date(2019, 8, 20),
 datetime.date(2024, 7, 2),
 datetime.date(2020, 2, 14),
 datetime.date(2022, 11, 1),
 datetime.date(2020, 2, 6),
 datetime.date(2020, 7, 9),
 datetime.date(2018, 11, 29),
 datetime.date(2022, 5, 9),
 datetime.date(2024, 7, 31),
 datetime.date(2022, 8, 1),
 datetime.date(2023, 4, 10),
 datetime.date(2023, 12, 6),
 datetime.date(2024

In [8]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [9]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [32]:
import pandas as pd

def rsi(series, length):
    delta = series.diff()
    
    gain = (delta.where(delta > 0, 0)).ewm(alpha=1/length, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/length, adjust=False).mean()

    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_signals(df, ema_length, rsi_length, rsi_ma_length):
    # Calculate X-day High/Low and 20 EMA
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Calculate RSI using TradingView's method
    df['RSI'] = rsi(df['close'], rsi_length)
    
    # Calculate RSI Moving Average
    df['RSI_MA'] = df['RSI'].rolling(rsi_ma_length).mean()

    # Sell signal condition
    df['Sell_Signal'] = df['RSI'] < df['RSI_MA']
    
    return df


In [35]:
def backtest(df, prev_n_candles):

    df.reset_index(drop=True, inplace=True)
    # Variables for short trades
    short_position = 0  # 0 = no position, 1 = short
    short_entry_price = 0
    short_entry_date = None
    short_trades = []
    tradebook = pd.DataFrame()
    tradebook_short = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    short_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_short = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for short trades
        short_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i-1, 'Sell_Signal'] and short_position == 0:
            #Signal is True on previous candle
            if df.loc[i, 'low'] < df.loc[i-1, 'low']:
                if df.loc[i, 'high'] < df.loc[i-1, 'low']:
                    # Gap Open Below Entry
                    continue
                else:
                    short_position = 1
                    short_entry_price = df.loc[i-1, 'low']
                    short_entry_date = df.loc[i, 'datetime']
                    short_trailing_stop = df.loc[i, 'EMA_20']
                    short_initial_sl = df.loc[i-prev_n_candles:i-1, 'high'].max()
        
                    tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
                        'Trade No.': trade_number,
                        'Entry_Date': df.loc[i, 'datetime'],
                        'Exit_Date': None,
                        'Trade_Type': 'short',
                        'Entry_Price': df.loc[i-1, 'low'],
                        # 'Previous MA Value': df.loc[i-1, 'X_Low'],
                        'Initial SL': short_initial_sl,
                        'Exit_Price': None,
                        # 'Profit/Loss': None  # Filled when exited
                    }])], ignore_index=True)
        
                    if not lock_initial_sl:
                        first_sl = short_initial_sl
                        lock_initial_sl = True
        
                    can_pyramid = True
                    pyramid_number = 0
                    pyramid_high = 0
                    continue
                
        # Exit or add to short position
        if short_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'high'] >= short_initial_sl:
                if df.loc[i, 'open'] >= short_initial_sl:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , df.loc[i, 'open'], 'Gap']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
                    # if df.loc[i, 'high'] >= long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , long_initial_sl, 'Initial SL Hit']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                    # elif df.loc[i, 'high'] < long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'], 'Gap Open-Close Below ISL']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                else:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , short_initial_sl, 'Initial SL Hit']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
            
            elif df.loc[i, 'close'] >= short_trailing_stop:
                tradebook_short.loc[
                    (tradebook_short['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                short_position = 0
                can_add_short = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            elif df.loc[i, 'RSI'] > df.loc[i, 'RSI_MA']:
                tradebook_short.loc[
                    (tradebook_short['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'RSI Upmove']
                short_position = 0
                can_add_short = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
    
    tradebook = pd.concat([tradebook_short, tradebook], ignore_index=True)
    return tradebook


In [45]:
ema_length = 20
RPT = 2
n_candle_sl = 5
rsi_length = 12
rsi_ma_length = 9
df = bnf_1min
df = calculate_signals(df, ema_length, rsi_length, rsi_ma_length)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest(df, n_candle_sl)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_33520/419236688.py:151: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RSI Upmove' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


In [46]:
import numpy as np
portfolio = 1000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio * 5 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [47]:
tb = tb.reset_index()

In [48]:
tb.tail(20)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
2384,2384,2384,2025-01-03 09:45:00+05:30,2025-01-03 11:15:00+05:30,short,51361.8500,51672.7500,51322.2500,Trailing SL Hit,39.6000,10.2684,29.3316,64.3294,1886.8826,0.1887,2025
2385,2385,2385,2025-01-03 12:15:00+05:30,2025-01-06 15:15:00+05:30,short,51243.5000,51439.1500,49960.2000,RSI Upmove,1283.3000,10.1204,1273.1796,102.2234,130148.6972,13.0149,2025
2386,2386,2386,2025-01-07 14:15:00+05:30,2025-01-08 13:15:00+05:30,short,50224.1000,50447.6000,49681.1500,RSI Upmove,542.9500,9.9905,532.9595,89.4855,47692.1230,4.7692,2025
2387,2387,2387,2025-01-09 10:45:00+05:30,2025-01-09 13:15:00+05:30,short,49569.8000,49927.7000,49378.7000,RSI Upmove,191.1000,9.8948,181.2052,55.8815,10126.0212,1.0126,2025
2388,2388,2388,2025-01-10 09:45:00+05:30,2025-01-10 10:45:00+05:30,short,49064.8000,49557.3500,49273.6500,RSI Upmove,-208.8500,9.8338,-218.6838,40.6050,-8879.6607,-0.8880,2025
2389,2389,2389,2025-01-10 11:45:00+05:30,2025-01-10 15:15:00+05:30,short,49061.5500,49483.1500,48779.8000,RSI Upmove,281.7500,9.7841,271.9659,47.4383,12901.6065,1.2902,2025
2390,2390,2390,2025-01-13 09:45:00+05:30,2025-01-13 10:15:00+05:30,short,48178.3500,48980.8000,48477.2500,RSI Upmove,-298.9000,9.6656,-308.5656,24.9237,-7690.5866,-0.7691,2025
2391,2391,2391,2025-01-13 12:15:00+05:30,2025-01-13 15:15:00+05:30,short,48251.4500,48606.3500,48047.3000,RSI Upmove,204.1500,9.6299,194.5201,56.3539,10961.9682,1.0962,2025
2392,2392,2392,2025-01-15 12:15:00+05:30,2025-01-15 12:45:00+05:30,short,48875.1000,49083.6500,48798.2000,Trailing SL Hit,76.9000,9.7673,67.1327,95.9003,6438.0408,0.6438,2025
2393,2393,2393,2025-01-15 13:15:00+05:30,2025-01-15 14:15:00+05:30,short,48792.0000,49083.6500,48767.6500,Trailing SL Hit,24.3500,9.7560,14.5940,68.5753,1000.7910,0.1001,2025


In [40]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [41]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [42]:
stats = generate_stats(tb, 'RSI Shorting')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,0,0,NaN,NaN,NaN,NaN,NaN,RSI Shorting
2018,-22.0851,335,34.3284,2.4460,-1.3790,-56.9805,-0.3876,RSI Shorting
2019,72.6448,305,39.0164,2.7857,-1.3917,-51.5890,1.4081,RSI Shorting
2020,77.5195,327,37.6147,2.8464,-1.3362,-44.5911,1.7385,RSI Shorting
2021,28.3751,342,38.0117,2.4815,-1.3878,-65.6171,0.4324,RSI Shorting
2022,-54.1877,368,39.1304,2.0451,-1.5566,-98.1237,-0.5522,RSI Shorting
2023,-37.2448,359,33.1476,2.4511,-1.3705,-99.7876,-0.3732,RSI Shorting
2024,-15.1339,345,35.0725,2.4606,-1.3967,-49.4690,-0.3059,RSI Shorting
2025,21.8512,23,65.2174,2.4235,-1.8126,-4.4239,4.9394,RSI Shorting
Overall,71.7392,2404,36.8552,2.4896,-1.4058,-170.5358,0.4207,RSI Shorting


In [ ]:
RPT_ = 3
stats_dictionary = {}

for ema_length in range(10, 81, 5):
    for rsi_length in range(6, 37, 3):
        for rsi_ma_length in range(5, 36, 5):
            for n1 in range(2, 11, 2):
                # RPT = 5
                variation = f'EMA:{ema_length}, RSI:{rsi_length}, RSI_MA:{rsi_ma_length}, SL Candles:{n1}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length, rsi_length, rsi_ma_length)
                tb = backtest(df, n1)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                    
                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 0:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                            stats_dictionary[x] = y
                        break

EMA:10, RSI:6, RSI_MA:5, SL Candles:2


/tmp/ipykernel_33520/419236688.py:137: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


EMA:10, RSI:6, RSI_MA:5, SL Candles:4


/tmp/ipykernel_33520/419236688.py:137: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


EMA:10, RSI:6, RSI_MA:5, SL Candles:6


...

/tmp/ipykernel_33520/419236688.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


EMA:10, RSI:24, RSI_MA:30, SL Candles:8


/tmp/ipykernel_33520/419236688.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                                Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:10, RSI:24, RSI_MA:30, SL Candles:8
2018     -47.0941          213  30.9859                3.2503             -1.7797     -49.5180      -0.9510  EMA:10, RSI:24, RSI_MA:30, SL Candles:8
2019     -10.5057          228  28.5088                4.1038             -1.7009     -64.0643      -0.1640  EMA:10, RSI:24, RSI_MA:30, SL Candles:8
2020      80.4653          205  35.1220                4.0989             -1.6139     -35.5770       2.2617  EMA:10, RSI:24, RSI_MA:30, SL Candles:8
2021       8.4086          221  31.2217                3.9337             -1.7304     -75.4910       0.1114  EMA:10, RSI:24, RSI_MA:30, SL Candles:8
2022     -18.4160          237  27.8481                3.8626             -1.5985     -79.4239      -0.231

/tmp/ipykernel_33520/419236688.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[
